In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import BaseCrossValidator
from sklearn.linear_model import Ridge
import time
from sklearn.metrics import make_scorer
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
df = pd.read_excel('Data diolah darin.xlsx')
df

,Backhoe,HP(watt),Norm_MAP,RPM,Age,Load_Factor,Engine_Tier,TEMP[C],Fuel[g/s],NOx[g/s],...,NOx[g/hr],HC[g/hr],CO[g/hr],CO2[g/hr],PM[mg/hr],Nox (g/kl),HC (g/kl),CO (g/kl),CO2 (g/kl),PM (g/kl)
0,1,65621.6,0.000000,833,12,0.21,2,22,0.430000,0.024090,...,86.724000,25.848000,13.176000,4780.656000,14.4,3730.117500,1111.757728,566.717728,2.056225e+05,0.619362
1,1,65621.6,0.012346,800,12,0.21,2,22,0.620000,0.030320,...,109.152000,25.668000,16.164000,7008.264000,21.6,4694.776362,1104.015681,695.235681,3.014350e+05,0.929047
2,1,65621.6,0.012346,826,12,0.21,2,22,0.510000,0.030730,...,110.628000,10.980000,10.584000,5722.920000,21.6,4758.261138,472.264772,455.232272,2.461506e+05,0.929047
3,1,65621.6,0.012346,831,12,0.21,2,22,0.390522,0.023693,...,85.293664,20.747853,4.825082,4382.553038,21.6,3668.596780,892.393432,207.533355,1.884996e+05,0.929047
4,1,65621.6,0.012346,834,12,0.21,2,22,0.413024,0.026107,...,93.985262,6.979907,9.880556,4673.503069,21.6,4042.434273,300.215326,424.976193,2.010137e+05,0.929047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37523,8,72332.9,0.975904,1676,36,0.21,2,41,1.590000,0.058830,...,211.788000,19.908000,28.152000,17957.592000,1800.0,445343.100000,41862.100000,59197.400000,3.776083e+07,3785.000000
37524,8,72332.9,0.975904,1683,36,0.21,2,42,1.320000,0.059500,...,214.200000,19.800000,30.780000,14905.044000,1656.0,542545.340900,50151.250000,77962.397730,3.775286e+07,4194.468182
37525,8,72332.9,0.975904,1846,36,0.21,2,42,1.530000,0.059640,...,214.704000,21.744000,33.840000,17349.048000,1872.0,469179.694100,47515.850980,73948.509800,3.791183e+07,4090.768627
37526,8,72332.9,0.987952,1876,36,0.21,2,42,1.760000,0.056200,...,202.320000,22.356000,34.740000,19948.572000,1980.0,384340.943200,42468.990340,65994.485800,3.789568e+07,3761.343750


In [ ]:
df.replace('#DIV/0!', np.nan, inplace=True)
df.dropna(inplace=True)

C:\Users\TI\AppData\Local\Temp\ipykernel_9016\2881342136.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('#DIV/0!', np.nan, inplace=True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37527 entries, 0 to 37527
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Backhoe      37527 non-null  int64  
 1   HP(watt)     37527 non-null  float64
 2   Norm_MAP     37527 non-null  float64
 3   RPM          37527 non-null  float64
 4   Age          37527 non-null  int64  
 5   Load_Factor  37527 non-null  float64
 6   Engine_Tier  37527 non-null  int64  
 7   TEMP[C]      37527 non-null  float64
 8   Fuel[g/s]    37527 non-null  float64
 9   NOx[g/s]     37527 non-null  float64
 10  HC[g/s]      37527 non-null  float64
 11  CO[g/s]      37527 non-null  float64
 12  CO2[g/s]     37527 non-null  float64
 13  PM[mg/s]     37527 non-null  float64
 14  Nox (g/l)    37527 non-null  float64
 15  HC (g/l)     37527 non-null  float64
 16  CO (g/l)     37527 non-null  float64
 17  CO2 (g/l)    37527 non-null  float64
 18  PM (g/l)     37527 non-null  float64
 19  Fuel[g/hr

In [ ]:
X = df[['Backhoe', 'HP(watt)', 'Norm_MAP', 'RPM', 'Age', 'Load_Factor', 'Engine_Tier']]
y = df['HC[g/s]']

In [ ]:
PredictorScaler = StandardScaler()
TargetVarScaler = StandardScaler()

PredictorScalerFit = PredictorScaler.fit(X)
TargetVarScalerFit = TargetVarScaler.fit(y.values.reshape(-1, 1))  # Convert y to numpy array and reshape

X = PredictorScalerFit.transform(X)
y = TargetVarScalerFit.transform(y.values.reshape(-1, 1))  # Convert y to numpy array and reshape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(26268, 7)
(26268,)
(11259, 7)
(11259,)


GBR MODEL

In [ ]:
gbr = GradientBoostingRegressor(loss='squared_error')

In [ ]:
gbr.fit(X_train,y_train)
y_pred = gbr.predict(X_test)

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mape(y_true, y_pred):
    return mean_absolute_percentage_error(y_true, y_pred)

rmse_score = rmse(y_test, y_pred)
mape_score = mape(y_test, y_pred)

print(f'RMSE: {rmse_score:.2f}')
print(f"MAPE: {mape_score:.2f}%")

RMSE: 0.73
MAPE: 1.92%


GRID

In [ ]:
params = {
    'learning_rate': np.arange(0.1, 1.0, 0.1),
    'n_estimators': np.arange(1, 51, 5),
    'subsample': np.arange(0.5, 1.0, 0.1),
    'min_samples_split': range(2, 11),
    'min_samples_leaf': range(1, 11),
    'max_depth': range(1, 26)
}

rmse_scorer = make_scorer(mean_squared_error, squared=False)
mape_scorer = make_scorer(mean_absolute_percentage_error)

In [ ]:
grid = GridSearchCV(
    estimator=gbr,
    param_grid=params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE',
    cv=2,
    verbose=3,
    n_jobs=-1
)

start_time = time.time()
gbr_result = grid.fit(X_train,y_train)

print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
print("Best RMSE:", grid.best_score_)
print("Best MAPE:", grid.cv_results_['mean_test_MAPE'][grid.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Fitting 2 folds for each of 1012500 candidates, totalling 2025000 fits
Best: 1.550329 using {'learning_rate': 0.9, 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 46, 'subsample': 0.5}
Best RMSE: 1.5503293074894087
Best MAPE: 6.761625698006611
Execution time: 52789.99315428734 s


In [ ]:
best_grid = grid.best_estimator_
y_pred_grid = best_grid.predict(X_test)
rmse_grid = np.sqrt(mean_squared_error(y_test, y_pred_grid))
mape_grid = mean_absolute_percentage_error(y_test, y_pred_grid)
print(f"RMSE: {rmse_grid}")
print(f"MAPE: {mape_grid:.2f}%")

RMSE: 1.5622704307673652
MAPE: 6.69%


In [ ]:
# export to excel
pd.DataFrame(grid.cv_results_).to_excel('GBR_GS_scal_HC.xlsx', index=False)

random search

In [ ]:
rs = RandomizedSearchCV(
    gbr,
    params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE',
    n_iter=100,
    cv=5,
    verbose=0,
    n_jobs=-1
)

start_time = time.time()
rs_result = rs.fit(X_train,y_train)

print("Best: %f using %s" % (rs.best_score_, rs.best_params_))
print("Best RMSE:", rs.best_score_)
print("Best MAPE:", rs.cv_results_['mean_test_MAPE'][rs.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Best: 1.483645 using {'subsample': 0.5, 'n_estimators': 46, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_depth': 24, 'learning_rate': 0.9}
Best RMSE: 1.4836451117694325
Best MAPE: 7.695292733197033
Execution time: 17.1321439743042 s


In [ ]:
best_rs = rs.best_estimator_
y_pred_rs = best_rs.predict(X_test)
rmse_rs = np.sqrt(mean_squared_error(y_test, y_pred_rs))
mape_rs = mean_absolute_percentage_error(y_test, y_pred_rs)
print(f"RMSE: {rmse_rs}")
print(f"MAPE: {mape_rs:.2f}%")

RMSE: 1.505743536352366
MAPE: 6.29%


In [ ]:
rs.cv_results_

{'mean_fit_time': array([0.20175176, 1.53007751, 0.53470068, 0.31459689, 0.01852508,
        0.33517365, 1.60240889, 0.13571153, 0.01451969, 0.12021284,
        0.48329391, 0.05613937, 0.38202171, 0.1803143 , 0.18726935,
        0.96749058, 0.46485496, 0.264358  , 0.37333274, 0.64153476,
        0.42507515, 1.51670299, 0.2729146 , 1.03216176, 0.01585236,
        0.09432869, 0.22448988, 0.84795208, 0.26864057, 0.59767303,
        0.84532075, 0.33411098, 1.32825675, 0.38197598, 0.02921047,
        0.29874039, 1.0700592 , 1.35187922, 0.39244027, 0.00778179,
        0.07062106, 1.75268812, 0.40425458, 0.67955747, 0.230936  ,
        0.30603085, 0.15799804, 1.10591173, 0.45681491, 0.35383525,
        0.8533164 , 0.03047123, 0.08198833, 0.75162005, 0.02777686,
        0.16773772, 1.89578013, 0.33954487, 0.55594501, 0.38996096,
        0.54581604, 2.04142079, 0.02445207, 1.59477015, 1.46149631,
        0.47881541, 0.15213113, 0.15812445, 1.61002388, 0.19367552,
        0.75158005, 0.11857347,

In [ ]:
pd.DataFrame(rs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_depth,param_learning_rate,...,std_test_RMSE,rank_test_RMSE,split0_test_MAPE,split1_test_MAPE,split2_test_MAPE,split3_test_MAPE,split4_test_MAPE,mean_test_MAPE,std_test_MAPE,rank_test_MAPE
0,0.201752,0.015234,0.003522,0.004260,0.8,36,10,4,1,0.5,...,0.025706,14,3.160986,3.244153,2.283555,3.150267,2.838571,2.935507,0.354023,35
1,1.530078,0.082690,0.025480,0.004720,0.9,26,2,1,24,0.7,...,0.020488,16,3.446018,4.484233,2.455434,4.489715,3.556760,3.686432,0.757855,13
2,0.534701,0.049152,0.004200,0.004217,0.8,16,8,6,13,0.6,...,0.012550,61,2.848428,3.734210,2.113539,3.337957,3.268568,3.060540,0.550612,30
3,0.314597,0.021013,0.002620,0.002490,0.8,6,5,1,21,0.8,...,0.015695,12,4.693038,4.611553,2.312583,3.793017,4.039004,3.889839,0.858529,5
4,0.018525,0.002458,0.000000,0.000000,0.5,1,9,3,7,0.8,...,0.012758,43,2.020650,2.739829,1.597147,2.067279,1.966620,2.078305,0.370148,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.020358,0.051575,0.016918,0.003492,0.5,31,8,3,16,0.3,...,0.021485,49,3.076949,3.378774,1.945575,3.597194,2.677230,2.935145,0.583265,36
96,1.504934,0.021532,0.017202,0.003215,0.8,26,2,4,24,0.7,...,0.023232,27,3.793594,4.583513,2.453281,4.069667,3.840069,3.748025,0.705465,7
97,0.518501,0.004622,0.004014,0.004926,0.9,41,3,9,3,0.5,...,0.019572,62,2.250852,2.790152,1.699186,2.605698,2.289595,2.327097,0.372433,82
98,1.609923,0.053328,0.020431,0.004132,0.9,26,3,1,24,0.8,...,0.024028,9,3.424990,4.242333,2.576489,3.428348,3.886058,3.511644,0.559116,18


In [ ]:
# export to excel
pd.DataFrame(rs.cv_results_).to_excel('GBR_RS_scal_HC.xlsx', index=False)

BAYESSIAN

In [ ]:
def gb_regression_cv(learning_rate,n_estimators, subsample, min_samples_split, min_samples_leaf, max_depth):
    gbr = GradientBoostingRegressor(
        learning_rate=learning_rate,
        n_estimators=int(n_estimators),
        subsample=(subsample),
        min_samples_split=int(min_samples_split),
        min_samples_leaf=int(min_samples_leaf),
        max_depth=int(max_depth)
    )
    scores = cross_val_score(gbr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    return -rmse  # Return only the RMSE score

In [ ]:
bayes_opt = BayesianOptimization(
    gb_regression_cv,
    {
        'learning_rate': (0.01, 1.0),
        'n_estimators': (1, 51),
        'subsample': (0.5, 1.0),
        'min_samples_split': (2, 11),
        'min_samples_leaf': (1, 11),
        'max_depth': (1, 26),
    }
)

In [ ]:
start_time = time.time()
bayes_opt.maximize(init_points=10, n_iter=100)
print("Execution time: " + str((time.time() - start_time)) + ' s')

|   iter    |  target   | learni... | max_depth | min_sa... | min_sa... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
| 1         | -0.6848   | 0.6411    | 6.561     | 8.378     | 6.623     | 12.51     | 0.9448    |
| 2         | -0.7074   | 0.7995    | 19.76     | 10.2      | 9.784     | 9.128     | 0.8364    |
| 3         | -0.6787   | 0.3273    | 22.56     | 10.52     | 4.348     | 15.39     | 0.5294    |
| 4         | -0.8191   | 0.8909    | 14.05     | 8.578     | 4.224     | 42.07     | 0.6377    |
| 5         | -0.6668   | 0.09573   | 22.13     | 7.721     | 5.835     | 38.33     | 0.6875    |
| 6         | -0.9394   | 0.0861    | 23.45     | 3.212     | 4.974     | 1.228     | 0.8236    |
| 7         | -0.8347   | 0.7826    | 1.11      | 2.612     | 9.467     | 23.5      | 0.7044    |
| 8         | -0.8545   | 0.4387    | 1.335     | 3.451     | 10.68     | 21.7      | 0.7751    |
| 9         | -0.752

In [ ]:
best_iter = np.argmin([res['target'] for res in bayes_opt.res])

In [ ]:
best_params = bayes_opt.res[best_iter]['params']

print("Best Parameters:", best_params)

Best Parameters: {'learning_rate': 0.01, 'max_depth': 7.421325720161643, 'min_samples_leaf': 11.0, 'min_samples_split': 6.579369619672955, 'n_estimators': 6.913311473586971, 'subsample': 0.5}


In [ ]:
best_bs = GradientBoostingRegressor(
    learning_rate=(best_params['learning_rate']),
    n_estimators=int(best_params['n_estimators']),
    subsample=(best_params['subsample']),
    min_samples_split=int(best_params['min_samples_split']),
    min_samples_leaf=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    loss='squared_error'
)
best_bs.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.01, max_depth=7, min_samples_leaf=6,
                          min_samples_split=6, n_estimators=6, subsample=0.5)

In [ ]:
y_pred_bs = best_bs.predict(X_test)
rmse_bs = np.sqrt(mean_squared_error(y_test, y_pred_bs))
mape_bs = mean_absolute_percentage_error(y_test, y_pred_bs)

print(f"RMSE: {rmse_bs}")
print(f"MAPE: {mape_bs:.2f}%")

RMSE: 0.9979576625401042
MAPE: 1.02%
